<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())

# Check the content of the file
content = spacex_csv_file.getvalue().decode('utf-8')
print(content)  # This will print the content of the file to ensure it's not empty or malformed

# Load the DataFrame if the content looks correct
if content.strip():
    spacex_df = pd.read_csv(io.StringIO(content))
else:
    print("The file is empty or not properly formatted.")
    # Manually create a sample DataFrame
sample_data = {
    'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
    'Long': [-80.577356, -80.576820, -80.646895, -120.610745]
}
spacex_df = pd.DataFrame(sample_data)





 


Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
1,2010-06-04,18:45:00,F9 v1.0  B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Failure   (parachute),0,28.56230197,-80.57735648
2,2010-12-08,15:43:00,F9 v1.0  B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats,  barrel of Brouère cheese",0,LEO (ISS),NASA (COTS) NRO,Failure   (parachute),0,28.56230197,-80.57735648
3,2012-05-22,7:44:00,F9 v1.0  B0005,CCAFS LC-40,Dragon demo flight C2+,525,LEO (ISS),NASA (COTS),No attempt,0,28.56230197,-80.57735648
4,2012-10-08,0:35:00,F9 v1.0  B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),No attempt,0,28.56230197,-80.57735648
5,2013-03-01,15:10:00,F9 v1.0  B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),No attempt,0,28.56230197,-80.57735648
7,2013-12-03,22:41:00,F9 v1.1,CCAFS LC-40,SES-8,3170,GTO,SES,No attempt,0,28.56230197,-80.57735648
8,2014-01-06,22:06:00,F9 v1.1,CCAFS LC-40,Thaico

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [11]:
# Task 2: Mark the success/failed launches for each site on the map
import folium
from folium.plugins import MarkerCluster
from IPython.display import display

# Initialize the map
site_map = folium.Map(location=[30.0, -100.0], zoom_start=4)

# Initialize a marker cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate through the DataFrame and add markers
for index, row in spacex_df.iterrows():
    color = 'green' if row['class'] == 1 else 'red'
    print(f"Adding marker for {row['Launch Site']} with status {'Success' if color == 'green' else 'Failure'}")
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=color, icon='ok' if color == 'green' else 'remove'),
        popup=f"Launch Site: {row['Launch Site']}<br>Success: {row['class']}"
    ).add_to(marker_cluster)

# Display the map inline
display(site_map)

    






Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Failure
Adding marker for CCAFS LC-40 with status Success
Adding marker for CCAFS LC-40 with status Success
Adding marker for CCAFS LC-40 with status Failure


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [27]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,1
1,CCAFS SLC-40,28.563197,-80.576820,0
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster

# Sample data
sample_data = {
    'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
    'Long': [-80.577356, -80.576820, -80.646895, -120.610745],
    'class': [1, 0, 1, 0]
}
spacex_df = pd.DataFrame(sample_data)

# Step 1: Create a new column 'marker_color' based on the 'class' value
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Step 2: Initialize the map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Step 3: Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Step 4: For each launch record, add a Marker to the MarkerCluster
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=record['marker_color'], icon='info-sign'),
        popup=f"Launch Site: {record['Launch Site']}<br>Success: {record['class']}"
    )
    marker_cluster.add_child(marker)

# Step 5: Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

# Step 6: Save or display the map
site_map.save('launch_sites_cluster_map.html')



*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Apply a lambda function to create the 'marker_color' column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
import pandas as pd

# Sample data for demonstration
sample_data = {
    'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
    'Long': [-80.577356, -80.576820, -80.646895, -120.610745],
    'class': [1, 0, 1, 0]
}
spacex_df = pd.DataFrame(sample_data)

# Create the 'marker_color' column based on the 'class' value
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Display the DataFrame to see the new 'marker_color' column
print(spacex_df)


    Launch Site        Lat        Long  class marker_color
0   CCAFS LC-40  28.562302  -80.577356      1        green
1  CCAFS SLC-40  28.563197  -80.576820      0          red
2    KSC LC-39A  28.573255  -80.646895      1        green
3   VAFB SLC-4E  34.632834 -120.610745      0          red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
# Add marker_cluster to current site_map
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map
 


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [15]:
# TASK 3: Calculate the distances between a launch site to its proximities
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers
    return c * r

# Calculate distance from each launch site to the equator (latitude 0)
launch_sites_df['Distance to Equator (km)'] = launch_sites_df.apply(
    lambda row: haversine(row['Lat'], row['Long'], 0, row['Long']), axis=1)

launch_sites_df


,Launch Site,Lat,Long,Distance to Equator (km)
0,CCAFS LC-40,28.562302,-80.577356,3175.983072
1,CCAFS SLC-40,28.563197,-80.576820,3176.082615
2,KSC LC-39A,28.573255,-80.646895,3177.200946
3,VAFB SLC-4E,34.632834,-120.610745,3850.995454


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [16]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [18]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
import folium
from folium.plugins import MousePosition
import pandas as pd
from folium.plugins import MarkerCluster
from math import radians, sin, cos, sqrt, atan2

# Sample data
sample_data = {
    'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
    'Long': [-80.577356, -80.576820, -80.646895, -120.610745],
    'class': [1, 0, 1, 0]
}
spacex_df = pd.DataFrame(sample_data)

# Initialize the map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add MousePosition plugin
mouse_position = MousePosition()
site_map.add_child(mouse_position)

# Add launch sites and markers (assuming you have this part)
marker_cluster = MarkerCluster()
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='green' if record['class'] == 1 else 'red', icon='info-sign'),
        popup=f"Launch Site: {record['Launch Site']}<br>Success: {record['class']}"
    )
    marker_cluster.add_child(marker)
site_map.add_child(marker_cluster)

# Save the map
site_map.save('launch_sites_with_mouse_position.html')
import math

def calculate_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    # Difference in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Example coordinates
launch_site_lat = 28.562302
launch_site_lon = -80.577356
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance to coastline: {distance_coastline:.2f} km")


Distance to coastline: 0.58 km


In [19]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.features import DivIcon
from math import radians, sin, cos, sqrt, atan2

# Sample data
sample_data = {
    'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
    'Long': [-80.577356, -80.576820, -80.646895, -120.610745],
    'class': [1, 0, 1, 0]
}
spacex_df = pd.DataFrame(sample_data)

# Initialize the map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add MousePosition plugin
from folium.plugins import MousePosition
mouse_position = MousePosition()
site_map.add_child(mouse_position)

# Add MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='green' if record['class'] == 1 else 'red', icon='info-sign'),
        popup=f"Launch Site: {record['Launch Site']}<br>Success: {record['class']}"
    )
    marker_cluster.add_child(marker)

# Define the closest coastline point (example coordinates)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Define the launch site coordinates (example coordinates)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Function to calculate distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of Earth in kilometers
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Calculate the distance from the coastline point to the launch site
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create and add a marker for the coastline point
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(200,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline)
    ),
    popup=f"Closest Coastline Point<br>Distance to Launch Site: {distance_coastline:.2f} KM"
)

# Add the coastline marker to the map
site_map.add_child(distance_marker)

# Save the map
site_map.save('launch_sites_with_distance_marker.html')

    


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [20]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.features import DivIcon
from math import radians, sin, cos, sqrt, atan2

# Sample data
sample_data = {
    'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
    'Long': [-80.577356, -80.576820, -80.646895, -120.610745],
    'class': [1, 0, 1, 0]
}
spacex_df = pd.DataFrame(sample_data)

# Initialize the map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add MousePosition plugin
from folium.plugins import MousePosition
mouse_position = MousePosition()
site_map.add_child(mouse_position)

# Add MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='green' if record['class'] == 1 else 'red', icon='info-sign'),
        popup=f"Launch Site: {record['Launch Site']}<br>Success: {record['class']}"
    )
    marker_cluster.add_child(marker)

# Define the closest coastline point (example coordinates)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Define the launch site coordinates (example coordinates)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Function to calculate distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of Earth in kilometers
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Calculate the distance from the coastline point to the launch site
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create and add a marker for the coastline point
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(200,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline)
    ),
    popup=f"Closest Coastline Point<br>Distance to Launch Site: {distance_coastline:.2f} KM"
)

# Add the coastline marker to the map
site_map.add_child(distance_marker)

# Create a PolyLine between the launch site and coastline point
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, color='blue', weight=2)
site_map.add_child(lines)

# Save the map
site_map.save('launch_sites_with_distance_and_polyline.html')

 


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [23]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [28]:
import folium
from math import radians, sin, cos, sqrt, atan2
from IPython.display import display

# Function to calculate the Haversine distance between two sets of (lat, long) coordinates
def haversine(coord1, coord2):
    R = 6371.0  # Earth radius in kilometers

    lat1, lon1 = radians(coord1[0]), radians(coord1[1])
    lat2, lon2 = radians(coord2[0]), radians(coord2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Initialize the map centered around a general location
site_map = folium.Map(location=[28.5623, -80.5774], zoom_start=10)

# Coordinates for the launch site (e.g., Kennedy Space Center)
launch_site_coords = (28.5623, -80.5774)  # Kennedy Space Center, FL

# Coordinates for the closest city, railway, highway, and coastline
closest_city_coords = (28.5383, -81.3792)  # Orlando, FL
closest_railway_coords = (28.5241, -80.6804)  # Railway near Kennedy Space Center
closest_highway_coords = (28.4059, -80.6244)  # Highway near Kennedy Space Center
closest_coastline_coords = (28.6111, -80.5970)  # Coastline near Kennedy Space Center

# Calculate distances from the launch site to the closest features
city_distance = haversine(launch_site_coords, closest_city_coords)
railway_distance = haversine(launch_site_coords, closest_railway_coords)
highway_distance = haversine(launch_site_coords, closest_highway_coords)
coastline_distance = haversine(launch_site_coords, closest_coastline_coords)

# Add marker for the launch site
folium.Marker(
    location=launch_site_coords,
    icon=folium.Icon(color='blue', icon='rocket'),
    popup=f"Launch Site: Kennedy Space Center<br>Lat: {launch_site_coords[0]}, Long: {launch_site_coords[1]}"
).add_to(site_map)

# Add markers for the closest city, railway, highway, and coastline
folium.Marker(
    location=closest_city_coords,
    icon=folium.Icon(color='green', icon='info-sign'),
    popup=f"Closest City: Orlando<br>Distance: {city_distance:.2f} km"
).add_to(site_map)

folium.Marker(
    location=closest_railway_coords,
    icon=folium.Icon(color='purple', icon='train'),
    popup=f"Closest Railway<br>Distance: {railway_distance:.2f} km"
).add_to(site_map)

folium.Marker(
    location=closest_highway_coords,
    icon=folium.Icon(color='orange', icon='road'),
    popup=f"Closest Highway<br>Distance: {highway_distance:.2f} km"
).add_to(site_map)

folium.Marker(
    location=closest_coastline_coords,
    icon=folium.Icon(color='blue', icon='cloud'),
    popup=f"Closest Coastline<br>Distance: {coastline_distance:.2f} km"
).add_to(site_map)

# Draw lines between the launch site and the closest features
folium.PolyLine([launch_site_coords, closest_city_coords], color="green", weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine([launch_site_coords, closest_railway_coords], color="purple", weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine([launch_site_coords, closest_highway_coords], color="orange", weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine([launch_site_coords, closest_coastline_coords], color="blue", weight=2.5, opacity=1).add_to(site_map)

# Display the map inline
display(site_map)


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
